# Building Fast Queries on a CSV

Data 'Laptops' that we will gonna use originally came from [here](https://www.kaggle.com/datasets/muhammetvarl/laptop-price). This is short description of rows:

* ID: A unique identifier for the laptop.
* Company: The name of the company that produces the laptop.
* Product: The name of the laptop.
* TypeName: The type of laptop.
* Inches: The size of the screen in inches.
* ScreenResolution: The resolution of the screen.
* CPU: The laptop CPU.
* RAM: The amount of RAM in the laptop.
* Memory: The size of the hard drive.
* GPU: The graphics card name.
* OpSys: The name of the operating system.
* Weight: The laptop weight.
* Price: The price of the laptop.

The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

Given a laptop id, find the corresponding data.
Given an amount of money, find whether there are two laptops whose total price is that given amount.
Identify all laptops whose price falls within a given budget.

In [1]:
import csv
import pandas as pd
import time
import random

In [2]:
!wget http://bioinf-mw.bihz.upwr.edu.pl/students-data/laptops.csv

--2023-05-23 10:41:07--  http://bioinf-mw.bihz.upwr.edu.pl/students-data/laptops.csv
Resolving bioinf-mw.bihz.upwr.edu.pl (bioinf-mw.bihz.upwr.edu.pl)... 156.17.187.238
Connecting to bioinf-mw.bihz.upwr.edu.pl (bioinf-mw.bihz.upwr.edu.pl)|156.17.187.238|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197908 (193K) [text/csv]
Saving to: ‘laptops.csv’

laptops.csv         100%[===================>] 193.27K   131KB/s    in 1.5s    

2023-05-23 10:41:09 (131 KB/s) - ‘laptops.csv’ saved [197908/197908]



In [3]:
with open('laptops.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
   
    header = next(reader)

    rows = [row for row in reader]

print(header)

for row in rows[:5]:
    print(row)


['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD',

In [4]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])

inventory = Inventory('laptops.csv')  
print(inventory.header)               
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


This code defines a class called "Inventory" for managing inventory data. The class has an initializer method (init) that takes a CSV file name as an argument. Inside the initializer method, it opens the CSV file, reads its contents using the csv module, and stores the header row in the "header" attribute. It then stores the remaining rows of the CSV file (excluding the header) in the "rows" attribute. Additionally, the code converts the last element of each row to an integer. An instance of the "Inventory" class is created using the file name 'laptops.csv'. Finally, it prints the inventory data header and the number of rows (excluding the header) in the data.

In [5]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if laptop_id == row[0]:
                return row
        return None

A new method called "get_laptop_from_id(self, laptop_id)" has been added. This method takes a laptop ID as an argument and is responsible for searching the inventory data for the corresponding row based on that ID.

Inside the "get_laptop_from_id(self, laptop_id)" method, there is a loop that iterates through all the rows in the "rows" attribute (inventory data). For each row, it checks if the laptop ID matches the value in the first column (index 0). If there is a match, the entire row is returned as the result. If there is no match, it returns None.

In [6]:
inventory = Inventory('laptops.csv')  
print(inventory.get_laptop_from_id('3362737')) 
print(inventory.get_laptop_from_id('3362736')) 

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


In [7]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
          self.id_to_row[row[0]] = row

    def get_laptop_from_id_fast(self, laptop_id):
      if laptop_id in self.id_to_row:
        return self.id_to_row[laptop_id]
      return None
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if laptop_id == row[0]:
                return row
        return None

A new dictionary attribute called "id_to_row" has been added to the class. This dictionary is used to map laptop IDs to their corresponding rows in the inventory data.

In the "init" method, a loop has been introduced to populate the "id_to_row" dictionary. For each row in the "rows" attribute, the laptop ID (found in the first column, index 0) is used as the key, and the entire row is assigned as the value in the dictionary.

A new method called "get_laptop_from_id_fast(self, laptop_id)" has been implemented. This method allows for faster retrieval of a laptop row by directly accessing the "id_to_row" dictionary. If the provided laptop ID exists in the dictionary, the corresponding row is returned. Otherwise, None is returned.

In [8]:
inventory = Inventory('laptops.csv')  
print(inventory.get_laptop_from_id_fast('3362737')) 
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


In [9]:
random_ids = str([random.randint(1000000, 9999999) for _ in range(10000)])

Generating 10 000 random values in range 1000000 - 9999999

In [10]:
total_time_no_dict = 0
for id in random_ids:
  start = time.time()
  inventory.get_laptop_from_id(id)
  end = time.time()
  result = end - start
  total_time_no_dict += result

In [11]:
total_time_dict = 0
for id in random_ids:
  start = time.time()
  inventory.get_laptop_from_id_fast(id)
  end = time.time()
  result = end - start
  total_time_dict += result

Measuring time for both methods.

In [16]:
dict_diff = total_time_no_dict/total_time_dict

In [17]:
inventory = Inventory('laptops.csv')  
print('Time for nondictionary method: ' + str(total_time_no_dict))
print('Time for dictionary method: ' + str(total_time_dict))
print('Dictionary method is ' + str(dict_diff) + ' times faster than the other')

Time for nondictionary method: 7.122251987457275
Time for dictionary method: 0.06778359413146973
Dictionary method is 105.07338949367757 times faster than the other



The method with the dictionary is much faster. The time complexity of the method without the dictionary is O(R), where R is the number of rows. In the second method, the time complexity is constant, O(1). This is because the algorithm creates a dictionary that stores information based on the laptop IDs. Although creating and storing the dictionary requires more space, it allows for a shorter time complexity. 

In [18]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
          self.id_to_row[row[0]] = row

    def get_laptop_from_id_fast(self, laptop_id):
      if laptop_id in self.id_to_row:
        return self.id_to_row[laptop_id]
      return None
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if laptop_id == row[0]:
                return row
        return None

    def check_promotion_dollars(self, dollars):
      for row in self.rows:
        if dollars == row[-1]:
          return True

      for i in range(len(self.rows)):
        for j in range(len(self.rows)):
          if self.rows[i][-1] + self.rows[j][-1] == dollars:
            return True
      return False  


A new method called "check_promotion_dollars(self, dollars)" has been added to the class. This method checks if there is a laptop with a given price or a combination of two laptops whose prices add up to the specified "dollars" amount.

The "check_promotion_dollars" method first iterates through each row in the "rows" attribute and checks if the price of any laptop matches the given "dollars" value. If a match is found, it returns True.

If no direct match is found, the method proceeds to nested loops. It iterates through all combinations of two laptops by using two nested loops. For each combination, it checks if the sum of their prices is equal to the given "dollars" value. If a match is found, it returns True.

If no match is found in both the direct checks and combinations, the method returns False.

In [19]:
inventory = Inventory('laptops.csv')  
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


In [20]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
          self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
          self.prices.add(row[-1])

    def get_laptop_from_id_fast(self, laptop_id):
      if laptop_id in self.id_to_row:
        return self.id_to_row[laptop_id]
      return None
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if laptop_id == row[0]:
                return row
        return None

    def check_promotion_dollars(self, dollars):
      for row in self.rows:
        if dollars == row[-1]:
          return True

      for i in range(len(self.rows)):
        for j in range(len(self.rows)):
          if self.rows[i][-1] + self.rows[j][-1] == dollars:
            return True
      return False  

    def check_promotion_dollars_fast(self, dollars):
      if dollars in self.prices:
        return True

      if self.rows[-1] + self.rows[-1] == dollars:
        return True
      return False


The changes made to the code introduce a new method for checking whether a given amount corresponds to the price of a laptop in our inventory or if the combined price of two laptops equals the specified amount. This time, we created a set to store the laptop prices. This reduces the algorithm's complexity, and in the new version ("check_promotion_dollars_fast"), the complexity is O(n) instead of O(N^2), where n is the number of laptops in the inventory.

In [21]:
inventory = Inventory('laptops.csv')  
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


In [22]:
prices = [random.randint(100,5000) for _ in range(1000)]

Generating list of random prices

In [23]:
inventory = Inventory('laptops.csv')  

In [24]:
total_time_no_set = 0
for price in prices:
  start = time.time()
  inventory.check_promotion_dollars(price)
  end = time.time()
  result = end - start
  total_time_no_set += result

In [25]:
total_time_set = 0
for price in prices:
  start = time.time()
  inventory.check_promotion_dollars_fast(price)
  end = time.time()
  result = end - start
  total_time_set += result

In [26]:
print('Time for method without set: ' + str(total_time_no_set))
print('Time for method with set: ' + str(total_time_set))

Time for method without set: 18.75588631629944
Time for method with set: 0.0035026073455810547


In [28]:
time_diff_promotion = total_time_no_set/total_time_set
print('Second method is '+ str(time_diff_promotion) + ' times faster')

Second method is 5354.835545572119 times faster


In [29]:
 def row_price(row):
   return row[-1]

class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
          self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
          self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price)


    def get_laptop_from_id_fast(self, laptop_id):
      if laptop_id in self.id_to_row:
        return self.id_to_row[laptop_id]
      return None
    
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if laptop_id == row[0]:
                return row
        return None

    def check_promotion_dollars(self, dollars):
      for row in self.rows:
        if dollars == row[-1]:
          return True

      for i in range(len(self.rows)):
        for j in range(len(self.rows)):
          if self.rows[i][-1] + self.rows[j][-1] == dollars:
            return True
      return False  

    def check_promotion_dollars_fast(self, dollars):
      if dollars in self.prices:
        return True

      if self.rows[-1] + self.rows[-1] == dollars:
        return True
      return False

    def find_first_laptop_more_expensive(self, target_price):
      range_start = 0
      range_end = len(self.rows_by_price) - 1
      while range_start <= range_end:
        range_middle = (range_end + range_start) // 2
        price = self.rows_by_price[range_middle][-1]
        if price > target_price:
            range_end = range_middle - 1
        else:
            range_start = range_middle + 1
      if range_start >= len(self.rows_by_price) or self.rows_by_price[range_start][-1] <= target_price:
        return -1
      return range_start



A new method called "find_first_laptop_more_expensive" has been added. This method uses binary search to find the index of the first laptop in the sorted "rows_by_price" attribute whose price is greater than the given "target_price". It returns the index if found, or -1 if no such laptop is found.

In [30]:
inventory = Inventory('laptops.csv')                     # Step 3            
print(inventory.find_first_laptop_more_expensive(1000))  # Step 4
print(inventory.find_first_laptop_more_expensive(10000))

683
-1
